In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" xformers

In [2]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 4096
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3.1-8b-instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.6: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Standard rank for SQL pattern learning
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
)

Unsloth 2025.9.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


In [6]:
from datasets import load_dataset
dataset = load_dataset("b-mc2/sql-create-context", split="train[:3000]")

README.md: 0.00B [00:00, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['answer', 'question', 'context'],
    num_rows: 3000
})

In [9]:
print(dataset[0]['question'])

How many heads of the departments are older than 56 ?


In [10]:
print(dataset[0]['answer'])

SELECT COUNT(*) FROM head WHERE age > 56


In [12]:
print(dataset[0]['context'])

CREATE TABLE head (age INTEGER)


In [13]:
def format_sql_generation(examples):
    """Format examples for SQL generation task"""
    texts = []

    for i in range(len(examples["question"])):
        context = examples["context"][i]  # Database schema
        question = examples["question"][i]  # Natural language question
        answer = examples["answer"][i]  # SQL query

        # Create comprehensive prompt
        prompt = f"""Given the database schema below, write a SQL query to answer the question.

Database Schema:
{context}

Question: {question}

SQL Query:"""

        # Llama-3.1 chat template format
        messages = [
            {"role": "system", "content": "You are a SQL expert. Generate accurate SQL queries based on database schemas and natural language questions."},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": answer}
        ]

        # Apply chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )

        texts.append(text)

    return {"text": texts}

In [14]:
formatted_dataset = dataset.map(
    format_sql_generation,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Formatting for SQL generation"
)

Formatting for SQL generation:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [15]:
formatted_dataset

Dataset({
    features: ['text'],
    num_rows: 3000
})

In [16]:
print(formatted_dataset['text'][0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a SQL expert. Generate accurate SQL queries based on database schemas and natural language questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the database schema below, write a SQL query to answer the question.

Database Schema:
CREATE TABLE head (age INTEGER)

Question: How many heads of the departments are older than 56 ?

SQL Query:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

SELECT COUNT(*) FROM head WHERE age > 56<|eot_id|>


In [17]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # Smaller batch for complex SQL reasoning
    gradient_accumulation_steps=8,  # Effective batch size: 8
    warmup_steps=10,
    max_steps=200,  # More steps for SQL pattern learning
    learning_rate=1e-4,  # Lower LR for stable SQL training
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="llama_sql_expert",
    save_strategy="no",
)

In [18]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [19]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
5,3.100700
10,2.328800
15,1.203500
20,0.776900
25,0.678800
30,0.578100
35,0.552600
40,0.523200
45,0.507800
50,0.470200


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=200, training_loss=0.5808562976121903, metrics={'train_runtime': 1489.2989, 'train_samples_per_second': 1.074, 'train_steps_per_second': 0.134, 'total_flos': 1.0239393893081088e+16, 'train_loss': 0.5808562976121903, 'epoch': 0.5333333333333333})

In [20]:
model.save_pretrained("llama_sql_generator")
tokenizer.save_pretrained("llama_sql_generator")

('llama_sql_generator/tokenizer_config.json',
 'llama_sql_generator/special_tokens_map.json',
 'llama_sql_generator/chat_template.jinja',
 'llama_sql_generator/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [22]:
def generate_sql(schema, question, max_tokens=300):
    """Generate SQL query from natural language question and database schema"""

    prompt = f"""Given the database schema below, write a SQL query to answer the question.

Database Schema:
{schema}

Question: {question}

SQL Query:"""

    messages = [
        {"role": "system", "content": "You are a SQL expert. Generate accurate SQL queries based on database schemas and natural language questions."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Extract generated tokens and decode
    generated_tokens = outputs[0][len(inputs['input_ids'][0]):]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response.strip()

In [23]:
schema1 = """
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100),
    city VARCHAR(50)
);

CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL(10,2),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
"""

In [24]:
questions1 = [
    "Find all customers from New York",
    "Get the total sales for each customer",
    "List orders placed in the last 30 days",
    "Find customers who have never placed an order"
]

print("🛒 E-COMMERCE DATABASE QUERIES:")
for i, question in enumerate(questions1, 1):
    print(f"\n📝 Query {i}: {question}")
    sql = generate_sql(schema1, question, 200)
    print(f"Generated SQL:\n{sql}")
    print("-" * 50)

🛒 E-COMMERCE DATABASE QUERIES:

📝 Query 1: Find all customers from New York
Generated SQL:
SELECT name FROM customers WHERE city = 'New York'
--------------------------------------------------

📝 Query 2: Get the total sales for each customer
Generated SQL:
SELECT T1.customer_id, SUM(T2.total_amount) FROM customers AS T1 JOIN orders AS T2 ON T1.customer_id = T2.customer_id GROUP BY T1.customer_id
--------------------------------------------------

📝 Query 3: List orders placed in the last 30 days
Generated SQL:
SELECT * FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY)
--------------------------------------------------

📝 Query 4: Find customers who have never placed an order
Generated SQL:
SELECT name, email, city FROM customers WHERE NOT customer_id IN (SELECT customer_id FROM orders)
--------------------------------------------------


In [25]:
schema2 = """
CREATE TABLE departments (
    dept_id INT PRIMARY KEY,
    dept_name VARCHAR(50),
    location VARCHAR(50)
);

CREATE TABLE employees (
    emp_id INT PRIMARY KEY,
    name VARCHAR(100),
    dept_id INT,
    salary DECIMAL(10,2),
    hire_date DATE,
    FOREIGN KEY (dept_id) REFERENCES departments(dept_id)
);
"""

questions2 = [
    "Find the highest paid employee in each department",
    "List employees hired in 2023",
    "Get the average salary by department",
    "Find departments with more than 10 employees"
]

print("\n👥 EMPLOYEE DATABASE QUERIES:")
for i, question in enumerate(questions2, 1):
    print(f"\n📝 Query {i}: {question}")
    sql = generate_sql(schema2, question, 200)
    print(f"Generated SQL:\n{sql}")
    print("-" * 50)


👥 EMPLOYEE DATABASE QUERIES:

📝 Query 1: Find the highest paid employee in each department
Generated SQL:
SELECT T1.name, T1.salary, T1.hire_date, T2.dept_name FROM employees AS T1 JOIN departments AS T2 ON T1.dept_id = T2.dept_id ORDER BY T1.salary DESC LIMIT 1
--------------------------------------------------

📝 Query 2: List employees hired in 2023
Generated SQL:
SELECT name FROM employees WHERE hire_date >= '2023-01-01' AND hire_date <= '2023-12-31'
--------------------------------------------------

📝 Query 3: Get the average salary by department
Generated SQL:
SELECT AVG(T1.salary), T2.dept_name FROM employees AS T1 JOIN departments AS T2 ON T1.dept_id = T2.dept_id GROUP BY T1.dept_id
--------------------------------------------------

📝 Query 4: Find departments with more than 10 employees
Generated SQL:
SELECT T1.dept_name FROM departments AS T1 JOIN employees AS T2 ON T1.dept_id = T2.dept_id GROUP BY T1.dept_name HAVING COUNT(*) > 10
-----------------------------------------

In [26]:
schema3 = """
CREATE TABLE books (
    book_id INT PRIMARY KEY,
    title VARCHAR(200),
    author VARCHAR(100),
    publication_year INT,
    genre VARCHAR(50)
);

CREATE TABLE members (
    member_id INT PRIMARY KEY,
    name VARCHAR(100),
    membership_date DATE
);

CREATE TABLE loans (
    loan_id INT PRIMARY KEY,
    book_id INT,
    member_id INT,
    loan_date DATE,
    return_date DATE,
    FOREIGN KEY (book_id) REFERENCES books(book_id),
    FOREIGN KEY (member_id) REFERENCES members(member_id)
);
"""

questions3 = [
    "Find all books written by Stephen King",
    "List members who have overdue books",
    "Get the most popular books by loan count",
    "Find books that have never been borrowed"
]

print("\n📚 LIBRARY DATABASE QUERIES:")
for i, question in enumerate(questions3, 1):
    print(f"\n📝 Query {i}: {question}")
    sql = generate_sql(schema3, question, 250)
    print(f"Generated SQL:\n{sql}")
    print("-" * 50)



📚 LIBRARY DATABASE QUERIES:

📝 Query 1: Find all books written by Stephen King
Generated SQL:
SELECT title FROM books WHERE author = "Stephen King"
--------------------------------------------------

📝 Query 2: List members who have overdue books
Generated SQL:
SELECT T1.name FROM members AS T1 JOIN loans AS T2 ON T1.member_id = T2.member_id WHERE T2.return_date < T2.loan_date
--------------------------------------------------

📝 Query 3: Get the most popular books by loan count
Generated SQL:
SELECT T1.title FROM books AS T1 JOIN loans AS T2 ON T1.book_id = T2.book_id GROUP BY T1.title ORDER BY COUNT(*) DESC LIMIT 1
--------------------------------------------------

📝 Query 4: Find books that have never been borrowed
Generated SQL:
SELECT title FROM books WHERE NOT book_id IN (SELECT book_id FROM loans)
--------------------------------------------------


In [27]:
schema4 = """
CREATE TABLE sales (
    sale_id INT PRIMARY KEY,
    product_name VARCHAR(100),
    category VARCHAR(50),
    sale_date DATE,
    quantity INT,
    price DECIMAL(10,2),
    region VARCHAR(50)
);
"""

complex_questions = [
    "Calculate monthly sales trends for the last year",
    "Find the top 5 products by revenue in each region",
    "Compare this year's sales to last year's by category"
]

print("\n📈 COMPLEX ANALYTICS QUERIES:")
for i, question in enumerate(complex_questions, 1):
    print(f"\n📝 Complex Query {i}: {question}")
    sql = generate_sql(schema4, question, 300)
    print(f"Generated SQL:\n{sql}")
    print("-" * 50)


📈 COMPLEX ANALYTICS QUERIES:

📝 Complex Query 1: Calculate monthly sales trends for the last year
Generated SQL:
SELECT SUM(T1.quantity), T1.region, MONTH(T1.sale_date), YEAR(T1.sale_date) FROM sales AS T1 GROUP BY T1.region, MONTH(T1.sale_date), YEAR(T1.sale_date)
--------------------------------------------------

📝 Complex Query 2: Find the top 5 products by revenue in each region
Generated SQL:
SELECT product_name, region, SUM(quantity * price) FROM sales GROUP BY product_name, region ORDER BY SUM(quantity * price) DESC LIMIT 5
--------------------------------------------------

📝 Complex Query 3: Compare this year's sales to last year's by category
Generated SQL:
SELECT T1.category, SUM(T1.quantity) FROM sales AS T1 JOIN sales AS T2 ON T1.category = T2.category WHERE T1.sale_date = (SELECT MAX(sale_date) FROM sales) AND T2.sale_date = (SELECT MIN(sale_date) FROM sales) GROUP BY T1.category
--------------------------------------------------


In [28]:
if torch.cuda.is_available():
    print(f"\n📊 GPU Memory Usage:")
    print(f"Allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"Reserved: {torch.cuda.memory_reserved()/1024**3:.2f} GB")



📊 GPU Memory Usage:
Allocated: 5.79 GB
Reserved: 6.27 GB
